In [132]:
import numpy as np
import pandas as pd
import os, math

from sklearn.svm import NuSVR
from sklearn.preprocessing import StandardScaler

In [133]:
# Read xlsx files
Y = pd.read_excel('BRCA.subsample.xlsx', sheet_name = 0)
X = pd.read_excel('LM22Signature.xlsx', sheet_name = 0)

In [134]:
# Normalize signature Matrix
mean = X.iloc[:, 1:].mean()
std = X.iloc[:, 1:].stack().std()
X.iloc[:, 1:] = (X.iloc[:, 1:] - mean.mean()) / std

scaler = StandardScaler()
Yn = scaler.fit(Y.iloc[:, 1:])

scaler.transform(Y.iloc[:, 1:])

array([[-3.69493128e-01, -3.07050356e-01, -3.52411466e-01,
        -4.23422773e-01, -3.30782637e-01],
       [ 1.95485917e-01,  6.56018718e-02, -2.99927866e-02,
        -1.64676886e-01, -2.80904191e-01],
       [-3.30042289e-01,  9.23971641e-03,  2.12639880e-02,
        -1.66200316e-01, -2.07169015e-01],
       [-3.77383295e-01,  2.94718219e+00, -3.60459860e-01,
        -4.23422773e-01, -3.83161290e-01],
       [ 2.21561273e+00,  7.72994427e-01,  4.21562723e+00,
         4.00301523e-01,  8.40404047e-01],
       [-2.79832131e-01, -2.35333421e-01, -2.63879127e-01,
        -2.83328399e-01, -2.80499130e-01],
       [-2.77680267e-01, -7.71883848e-02, -2.45482797e-01,
        -2.83328399e-01, -3.52432480e-01],
       [-3.70927704e-01, -3.35246758e-01, -3.57010548e-01,
        -3.93566595e-01, -3.71288795e-01],
       [-3.70210416e-01, -3.29117105e-01, -3.37464447e-01,
        -3.59882702e-01, -3.26592345e-01],
       [-9.33372582e-02,  7.29881031e-02,  9.82986199e-02,
         3.09967446e-01

In [126]:
# Intersection between X and Y
X = X.loc[X['Gene symbol'].isin(Y['Gene symbol'])]
Y = Y.loc[Y['Gene symbol'].isin(X['Gene symbol'])]

In [128]:
# Ordering by GeneSymbol
X = X.sort_values(by=['Gene symbol'])
Y = Y.sort_values(by=['Gene symbol'])

# Slice Gene symbol column
X = X.iloc[:, 1:]
Y = Y.iloc[:, 1:]
X.reset_index(drop=True, inplace=True)

KeyError: 'Gene symbol'

In [88]:
# Run NuSVR
clf = NuSVR(kernel='linear', C=1.0, nu=0.1)
clf.fit(X, Y.iloc[:,2])


#for column in y:
#    clf = NuSVR(gamma='scale', kernel='linear', C=1.0, nu=0.1)
#    clf.fit(x, y[column])

NuSVR(C=1.0, cache_size=200, coef0=0.0, degree=3, gamma='auto_deprecated',
   kernel='linear', max_iter=-1, nu=0.1, shrinking=True, tol=0.001,
   verbose=False)

In [89]:
# Get betas
w = clf.coef_

# Set values i to zero where i < 0
w = np.where(w<0, 0, w)

# Normalize data
w = w/np.sum(w)

# Get delta (it will a parameter)
delta = 0.007

# Set values i to zero where i < delta
w = np.where(w<delta, 0, w)

In [91]:
# Product betas per rows X
neww = X.apply(lambda row: row*w[0], axis=1)

In [93]:
# Get Predict
predict = neww.sum(axis=1)

In [95]:
# Get Rmse predict
def getRmsePredict (x):
    dif = Y.apply(lambda col: col-predict, axis=0)
    math.sqrt(pow(dif,2).mean().mean())


38.1265860228339